<a href="https://colab.research.google.com/github/aliwilson2000/OptiIIProject1/blob/main/TransformerTestingProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers for MNIST

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
from google.colab import drive # mounting drive for easy read-write
drive.mount('/content/drive')

Mounted at /content/drive


### ViT Model 1:

In [ ]:
# this is written as a tensorflow "layer".  it's just a vector the same size as the
# output of the previous layer. the vector is initialized randomly, but we'll use
# gradient descent to update the values in the vector
#
# it's purpose is to be appended to the beginning of the sequence of vectors fed into
# the transformer.  then after the transformer runs on the whole data, we just grab
# the resulting zero-th vector...the class token...and use that as the portfolio weights
class ClassToken(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
            trainable = True
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        hidden_dim = self.w.shape[-1]

        cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
        cls = tf.cast(cls, dtype=inputs.dtype)
        return cls

In [ ]:
def build_ViT(n,m,block_size,hidden_dim,num_layers,num_heads,key_dim,mlp_dim,dropout_rate,num_classes):
    # n is number of rows of blocks
    # m is number of cols of blocks
    # block_size is number of pixels (with rgb) in each block

    inp = tf.keras.layers.Input(shape=(n*m,block_size))
    inp2 = tf.keras.layers.Input(shape=(n*m))
    mid = tf.keras.layers.Dense(hidden_dim)(inp) # transform to vectors with different dimension
    # the positional embeddings
#     positions = tf.range(start=0, limit=n*m, delta=1)
    emb = tf.keras.layers.Embedding(input_dim=n*m, output_dim=hidden_dim)(inp2)
    # learned positional embedding for each of the n*m possible possitions
    mid = mid + emb # for some reason, tf.keras.layers.Add causes an error, but + doesn't?
    # create and append class token to beginning of all input vectors
    token = ClassToken()(mid) # append class token to beginning of sequence
    mid = tf.keras.layers.Concatenate(axis=1)([token, mid])

    for l in range(num_layers): # how many Transformer Head layers are there?
        ln  = tf.keras.layers.LayerNormalization()(mid) # normalize
        mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,key_dim=key_dim,value_dim=key_dim)(ln,ln,ln)
        # self attention!
        add = tf.keras.layers.Add()([mid,mha]) # add and norm
        ln  = tf.keras.layers.LayerNormalization()(add)
        den = tf.keras.layers.Dense(mlp_dim,activation='gelu')(ln) # maybe should be relu...who knows...
        den = tf.keras.layers.Dropout(dropout_rate)(den) # regularization
        den = tf.keras.layers.Dense(hidden_dim)(den) # back to the right dimensional space
        den = tf.keras.layers.Dropout(dropout_rate)(den)
        mid = tf.keras.layers.Add()([den,add]) # add and norm again
    ln = tf.keras.layers.LayerNormalization()(mid)
    fl = ln[:,0,:] # just grab the class token for each image in batch
    clas = tf.keras.layers.Dense(num_classes,activation='softmax')(fl)
    # probability that the image is in each category
    mod = tf.keras.models.Model([inp,inp2],clas)
    mod.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return mod

In [ ]:
n = 4
m = 4
block_size = 49
hidden_dim = 128
num_layers = 16
num_heads = 8
key_dim = hidden_dim//num_heads
mlp_dim = hidden_dim
dropout_rate = 0.01
num_classes = 10



trans = build_ViT(n,m,block_size,hidden_dim,num_layers,num_heads,key_dim,mlp_dim,dropout_rate,num_classes)
trans.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 16, 49)]             0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 16, 128)              6400      ['input_1[0][0]']             
                                                                                                  
 embedding (Embedding)       (None, 16, 128)              2048      ['input_2[0][0]']             
                                                                                              

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

ndata_train = x_train.shape[0]
ndata_test = x_test.shape[0]

Preparing train and test data and flattening it to be fed to the transformer.

In [ ]:
x_train_ravel = np.zeros((ndata_train,n*m,block_size))
for img in range(ndata_train):
    ind = 0
    for row in range(n):
        for col in range(m):
            x_train_ravel[img,ind,:] = x_train[img,(row*7):((row+1)*7),(col*7):((col+1)*7)].ravel()
            ind += 1


In [ ]:
x_test_ravel = np.zeros((ndata_test,n*m,block_size))
for img in range(ndata_test):
    ind = 0
    for row in range(n):
        for col in range(m):
            x_test_ravel[img,ind,:] = x_test[img,(row*7):((row+1)*7),(col*7):((col+1)*7)].ravel()
            ind += 1

In [ ]:
pos_feed_train = np.array([list(range(n*m))]*ndata_train)
pos_feed_test = np.array([list(range(n*m))]*ndata_test)

In [ ]:
# Define your learning rate scheduler function
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr  # Keep the initial learning rate for the first 10 epochs
    else:
        return lr * tf.math.exp(-0.1)  # Exponentially decay the learning rate after 10 epochs

# Create the learning rate scheduler callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

# Define the model checkpoint callback to save the best model weights
checkpoint_filepath = 'model_checkpoint.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Call trans.fit() with the learning rate scheduler and model checkpoint callbacks
trans.fit([x_train_ravel, pos_feed_train], y_train,
          epochs=100, batch_size=256, validation_split=0.20,
          callbacks=[lr_callback, model_checkpoint_callback])

Epoch 1/100
188/188 [==============================] - 66s 117ms/step - loss: 0.9813 - accuracy: 0.6681 - val_loss: 0.2427 - val_accuracy: 0.9226 - lr: 0.0010
Epoch 2/100
188/188 [==============================] - 20s 104ms/step - loss: 0.1810 - accuracy: 0.9434 - val_loss: 0.1362 - val_accuracy: 0.9594 - lr: 0.0010
Epoch 3/100
188/188 [==============================] - 19s 103ms/step - loss: 0.1067 - accuracy: 0.9666 - val_loss: 0.1089 - val_accuracy: 0.9674 - lr: 0.0010
Epoch 4/100
188/188 [==============================] - 19s 102ms/step - loss: 0.0793 - accuracy: 0.9750 - val_loss: 0.0999 - val_accuracy: 0.9709 - lr: 0.0010
Epoch 5/100
188/188 [==============================] - 19s 103ms/step - loss: 0.0671 - accuracy: 0.9782 - val_loss: 0.0979 - val_accuracy: 0.9718 - lr: 0.0010
Epoch 6/100
188/188 [==============================] - 20s 107ms/step - loss: 0.0563 - accuracy: 0.9812 - val_loss: 0.0881 - val_accuracy: 0.9729 - lr: 0.0010
Epoch 7/100
188/188 [=========================

In [ ]:
#This code will be used for the best transformer (i.e. highest accuracy): saving the model for reproducibility

#trans.save("/content/drive/MyDrive/optimization_hw_1_t.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
out = trans.evaluate([x_test_ravel,pos_feed_test],y_test)

313/313 [==============================] - 10s 32ms/step - loss: 0.0705 - accuracy: 0.9880


In [ ]:
out[1]

0.9879999756813049

In [ ]:
x_train_ravel.shape

(60000, 16, 49)

### ViT Model 2:

Changing the Architecture from ViT 1 to ViT 2:
- Increase hidden dimensions slightly (128 to 144) - may increase overfitting byt the model will have a better capacity to learn features
- Reduce the number of layers from 16 t 8 (try and offset overfitting this way since the MNIST is not super complex)
- Increase the dropout rate from 0.01 to 0.08

In [ ]:
class ClassToken(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
            trainable = True
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        hidden_dim = self.w.shape[-1]

        cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
        cls = tf.cast(cls, dtype=inputs.dtype)
        return cls

In [ ]:
def build_ViT2(n,m,block_size,hidden_dim,num_layers,num_heads,key_dim,mlp_dim,dropout_rate,num_classes):
    inp = tf.keras.layers.Input(shape=(n*m,block_size))
    inp2 = tf.keras.layers.Input(shape=(n*m))
    mid = tf.keras.layers.Dense(hidden_dim)(inp)
    emb = tf.keras.layers.Embedding(input_dim=n*m, output_dim=hidden_dim)(inp2)
    mid = mid + emb
    token = ClassToken()(mid)
    mid = tf.keras.layers.Concatenate(axis=1)([token, mid])

    for l in range(num_layers):
        ln  = tf.keras.layers.LayerNormalization()(mid)
        mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,key_dim=key_dim,value_dim=key_dim)(ln,ln,ln)
        add = tf.keras.layers.Add()([mid,mha])
        ln  = tf.keras.layers.LayerNormalization()(add)
        den = tf.keras.layers.Dense(mlp_dim,activation='gelu')(ln)
        den = tf.keras.layers.Dropout(dropout_rate)(den)
        den = tf.keras.layers.Dense(hidden_dim)(den)
        den = tf.keras.layers.Dropout(dropout_rate)(den)
        mid = tf.keras.layers.Add()([den,add])
    ln = tf.keras.layers.LayerNormalization()(mid)
    fl = ln[:,0,:]
    clas = tf.keras.layers.Dense(num_classes,activation='softmax')(fl)
    mod = tf.keras.models.Model([inp,inp2],clas)
    mod.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return mod

In [ ]:
n = 4
m = 4
block_size = 49
hidden_dim = 144
num_layers = 8
num_heads = 8
key_dim = hidden_dim//num_heads
mlp_dim = hidden_dim
dropout_rate = 0.08
num_classes = 10

trans = build_ViT2(n,m,block_size,hidden_dim,num_layers,num_heads,key_dim,mlp_dim,dropout_rate,num_classes)
trans.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 16, 49)]             0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 16, 144)              7200      ['input_1[0][0]']             
                                                                                                  
 embedding (Embedding)       (None, 16, 144)              2304      ['input_2[0][0]']             
                                                                                              

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

ndata_train = x_train.shape[0]
ndata_test = x_test.shape[0]

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train_ravel = np.zeros((ndata_train,n*m,block_size))
for img in range(ndata_train):
    ind = 0
    for row in range(n):
        for col in range(m):
            x_train_ravel[img,ind,:] = x_train[img,(row*7):((row+1)*7),(col*7):((col+1)*7)].ravel()
            ind += 1

In [ ]:
x_test_ravel = np.zeros((ndata_test,n*m,block_size))
for img in range(ndata_test):
    ind = 0
    for row in range(n):
        for col in range(m):
            x_test_ravel[img,ind,:] = x_test[img,(row*7):((row+1)*7),(col*7):((col+1)*7)].ravel()
            ind += 1

In [ ]:
pos_feed_train = np.array([list(range(n*m))]*ndata_train)
pos_feed_test = np.array([list(range(n*m))]*ndata_test)

In [ ]:
## The training procedure and additional callbacks will be KEPT THE SAME for Model 1 to Model 2:
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

checkpoint_filepath = 'model_checkpoint.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

trans.fit([x_train_ravel, pos_feed_train], y_train,
          epochs=100, batch_size=256, validation_split=0.20,
          callbacks=[lr_callback, model_checkpoint_callback])

Epoch 1/100
188/188 [==============================] - 13s 70ms/step - loss: 0.1961 - accuracy: 0.9390 - val_loss: 0.1295 - val_accuracy: 0.9628 - lr: 0.0010
Epoch 2/100
188/188 [==============================] - 15s 80ms/step - loss: 0.1232 - accuracy: 0.9611 - val_loss: 0.1061 - val_accuracy: 0.9698 - lr: 0.0010
Epoch 3/100
188/188 [==============================] - 12s 63ms/step - loss: 0.0979 - accuracy: 0.9687 - val_loss: 0.0981 - val_accuracy: 0.9714 - lr: 0.0010
Epoch 4/100
188/188 [==============================] - 13s 67ms/step - loss: 0.0786 - accuracy: 0.9754 - val_loss: 0.0967 - val_accuracy: 0.9725 - lr: 0.0010
Epoch 5/100
188/188 [==============================] - 13s 71ms/step - loss: 0.0680 - accuracy: 0.9787 - val_loss: 0.0868 - val_accuracy: 0.9750 - lr: 0.0010
Epoch 6/100
188/188 [==============================] - 12s 62ms/step - loss: 0.0624 - accuracy: 0.9799 - val_loss: 0.0862 - val_accuracy: 0.9757 - lr: 0.0010
Epoch 7/100
188/188 [==============================]

In [ ]:
out = trans.evaluate([x_test_ravel,pos_feed_test],y_test)

313/313 [==============================] - 5s 17ms/step - loss: 0.0684 - accuracy: 0.9866


In [ ]:
out[1]

0.9865999817848206

In [ ]:
x_train_ravel.shape

(60000, 16, 49)

### ViT Model 3:

We are going to revert back to a dropout rate of 0.01 here and see what changes. Aside from this minor change everything is the same as Model 2 (in other words we have kept the increased hidden dimensions and reduced layers).

In [ ]:
class ClassToken(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
            trainable = True
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        hidden_dim = self.w.shape[-1]

        cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
        cls = tf.cast(cls, dtype=inputs.dtype)
        return cls

In [ ]:
def build_ViT3(n,m,block_size,hidden_dim,num_layers,num_heads,key_dim,mlp_dim,dropout_rate,num_classes):
    inp = tf.keras.layers.Input(shape=(n*m,block_size))
    inp2 = tf.keras.layers.Input(shape=(n*m))
    mid = tf.keras.layers.Dense(hidden_dim)(inp)
    emb = tf.keras.layers.Embedding(input_dim=n*m, output_dim=hidden_dim)(inp2)
    mid = mid + emb
    token = ClassToken()(mid)
    mid = tf.keras.layers.Concatenate(axis=1)([token, mid])

    for l in range(num_layers):
        ln  = tf.keras.layers.LayerNormalization()(mid)
        mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,key_dim=key_dim,value_dim=key_dim)(ln,ln,ln)
        add = tf.keras.layers.Add()([mid,mha])
        ln  = tf.keras.layers.LayerNormalization()(add)
        den = tf.keras.layers.Dense(mlp_dim,activation='gelu')(ln)
        den = tf.keras.layers.Dropout(dropout_rate)(den)
        den = tf.keras.layers.Dense(hidden_dim)(den)
        den = tf.keras.layers.Dropout(dropout_rate)(den)
        mid = tf.keras.layers.Add()([den,add])
    ln = tf.keras.layers.LayerNormalization()(mid)
    fl = ln[:,0,:]
    clas = tf.keras.layers.Dense(num_classes,activation='softmax')(fl)
    mod = tf.keras.models.Model([inp,inp2],clas)
    mod.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return mod

In [ ]:
n = 4
m = 4
block_size = 49
hidden_dim = 144
num_layers = 8
num_heads = 8
key_dim = hidden_dim//num_heads
mlp_dim = hidden_dim
dropout_rate = 0.01
num_classes = 10

trans = build_ViT3(n,m,block_size,hidden_dim,num_layers,num_heads,key_dim,mlp_dim,dropout_rate,num_classes)
trans.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 16, 49)]             0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 16, 144)              7200      ['input_1[0][0]']             
                                                                                                  
 embedding (Embedding)       (None, 16, 144)              2304      ['input_2[0][0]']             
                                                                                              

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

ndata_train = x_train.shape[0]
ndata_test = x_test.shape[0]

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train_ravel = np.zeros((ndata_train,n*m,block_size))
for img in range(ndata_train):
    ind = 0
    for row in range(n):
        for col in range(m):
            x_train_ravel[img,ind,:] = x_train[img,(row*7):((row+1)*7),(col*7):((col+1)*7)].ravel()
            ind += 1

In [ ]:
x_test_ravel = np.zeros((ndata_test,n*m,block_size))
for img in range(ndata_test):
    ind = 0
    for row in range(n):
        for col in range(m):
            x_test_ravel[img,ind,:] = x_test[img,(row*7):((row+1)*7),(col*7):((col+1)*7)].ravel()
            ind += 1

In [ ]:
pos_feed_train = np.array([list(range(n*m))]*ndata_train)
pos_feed_test = np.array([list(range(n*m))]*ndata_test)

In [ ]:
## The training procedure and additional callbacks will be KEPT THE SAME AGAIN
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

checkpoint_filepath = 'model_checkpoint.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

trans.fit([x_train_ravel, pos_feed_train], y_train,
          epochs=100, batch_size=256, validation_split=0.20,
          callbacks=[lr_callback, model_checkpoint_callback])

Epoch 1/100
188/188 [==============================] - 40s 74ms/step - loss: 0.9608 - accuracy: 0.6755 - val_loss: 0.2357 - val_accuracy: 0.9293 - lr: 0.0010
Epoch 2/100
188/188 [==============================] - 12s 66ms/step - loss: 0.1637 - accuracy: 0.9496 - val_loss: 0.1358 - val_accuracy: 0.9601 - lr: 0.0010
Epoch 3/100
188/188 [==============================] - 12s 66ms/step - loss: 0.1036 - accuracy: 0.9677 - val_loss: 0.1257 - val_accuracy: 0.9634 - lr: 0.0010
Epoch 4/100
188/188 [==============================] - 13s 68ms/step - loss: 0.0785 - accuracy: 0.9746 - val_loss: 0.1147 - val_accuracy: 0.9674 - lr: 0.0010
Epoch 5/100
188/188 [==============================] - 13s 67ms/step - loss: 0.0618 - accuracy: 0.9801 - val_loss: 0.1058 - val_accuracy: 0.9704 - lr: 0.0010
Epoch 6/100
188/188 [==============================] - 13s 67ms/step - loss: 0.0574 - accuracy: 0.9808 - val_loss: 0.0976 - val_accuracy: 0.9721 - lr: 0.0010
Epoch 7/100
188/188 [==============================]

In [ ]:
# saving the model for reproducibility

trans.save("/content/drive/MyDrive/transformer_3.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
out = trans.evaluate([x_test_ravel,pos_feed_test],y_test)

313/313 [==============================] - 7s 22ms/step - loss: 0.0769 - accuracy: 0.9861


In [ ]:
out[1]

0.9861000180244446

In [ ]:
x_train_ravel.shape

(60000, 16, 49)

Model 3 has an accuracy of 98.6% which is lower

### ViT Model 4:

Changing the Architecture from ViT 1 to ViT 4: Here we are again only going to make a slight change. Since we saw that the accuracy drops slightly with the dropout rate going from 0.08 to 0.01 (Model 2 to Model 3), we are going to increase this dropout rate back to 0.08 and then return to the structure of Model 1. In essence, Model 4 is Model 1 with a higher dropout rate.

In [ ]:
# this is written as a tensorflow "layer".  it's just a vector the same size as the
# output of the previous layer. the vector is initialized randomly, but we'll use
# gradient descent to update the values in the vector
#
# it's purpose is to be appended to the beginning of the sequence of vectors fed into
# the transformer.  then after the transformer runs on the whole data, we just grab
# the resulting zero-th vector...the class token...and use that as the portfolio weights
class ClassToken(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
            trainable = True
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        hidden_dim = self.w.shape[-1]

        cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
        cls = tf.cast(cls, dtype=inputs.dtype)
        return cls

In [ ]:
def build_ViT4(n,m,block_size,hidden_dim,num_layers,num_heads,key_dim,mlp_dim,dropout_rate,num_classes):
    # n is number of rows of blocks
    # m is number of cols of blocks
    # block_size is number of pixels (with rgb) in each block

    inp = tf.keras.layers.Input(shape=(n*m,block_size))
    inp2 = tf.keras.layers.Input(shape=(n*m))
    mid = tf.keras.layers.Dense(hidden_dim)(inp) # transform to vectors with different dimension
    # the positional embeddings
#     positions = tf.range(start=0, limit=n*m, delta=1)
    emb = tf.keras.layers.Embedding(input_dim=n*m, output_dim=hidden_dim)(inp2) # learned positional embedding for each of the n*m possible possitions
    mid = mid + emb # for some reason, tf.keras.layers.Add causes an error, but + doesn't?
    # create and append class token to beginning of all input vectors
    token = ClassToken()(mid) # append class token to beginning of sequence
    mid = tf.keras.layers.Concatenate(axis=1)([token, mid])

    for l in range(num_layers): # how many Transformer Head layers are there?
        ln  = tf.keras.layers.LayerNormalization()(mid) # normalize
        mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,key_dim=key_dim,value_dim=key_dim)(ln,ln,ln) # self attention!
        add = tf.keras.layers.Add()([mid,mha]) # add and norm
        ln  = tf.keras.layers.LayerNormalization()(add)
        den = tf.keras.layers.Dense(mlp_dim,activation='gelu')(ln) # maybe should be relu...who knows...
        den = tf.keras.layers.Dropout(dropout_rate)(den) # regularization
        den = tf.keras.layers.Dense(hidden_dim)(den) # back to the right dimensional space
        den = tf.keras.layers.Dropout(dropout_rate)(den)
        mid = tf.keras.layers.Add()([den,add]) # add and norm again
    ln = tf.keras.layers.LayerNormalization()(mid)
    fl = ln[:,0,:] # just grab the class token for each image in batch
    clas = tf.keras.layers.Dense(num_classes,activation='softmax')(fl) # probability that the image is in each category
    mod = tf.keras.models.Model([inp,inp2],clas)
    mod.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return mod

In [ ]:
n = 4
m = 4
block_size = 49
hidden_dim = 128
num_layers = 16
num_heads = 8
key_dim = hidden_dim//num_heads # usually good practice for key_dim to be hidden_dim//num_heads...this is why we do Multi-Head attention
mlp_dim = hidden_dim
dropout_rate = 0.08
num_classes = 10

trans = build_ViT4(n,m,block_size,hidden_dim,num_layers,num_heads,key_dim,mlp_dim,dropout_rate,num_classes)
trans.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 16, 49)]             0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 16, 128)              6400      ['input_1[0][0]']             
                                                                                                  
 embedding (Embedding)       (None, 16, 128)              2048      ['input_2[0][0]']             
                                                                                              

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

ndata_train = x_train.shape[0]
ndata_test = x_test.shape[0]

11490434/11490434 [==============================] - 2s 0us/step


In [ ]:
x_train_ravel = np.zeros((ndata_train,n*m,block_size))
for img in range(ndata_train):
    ind = 0
    for row in range(n):
        for col in range(m):
            x_train_ravel[img,ind,:] = x_train[img,(row*7):((row+1)*7),(col*7):((col+1)*7)].ravel()
            ind += 1

In [ ]:
x_test_ravel = np.zeros((ndata_test,n*m,block_size))
for img in range(ndata_test):
    ind = 0
    for row in range(n):
        for col in range(m):
            x_test_ravel[img,ind,:] = x_test[img,(row*7):((row+1)*7),(col*7):((col+1)*7)].ravel()
            ind += 1

In [ ]:
pos_feed_train = np.array([list(range(n*m))]*ndata_train)
pos_feed_test = np.array([list(range(n*m))]*ndata_test)

In [ ]:
# Define your learning rate scheduler function
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr  # Keep the initial learning rate for the first 10 epochs
    else:
        return lr * tf.math.exp(-0.1)  # Exponentially decay the learning rate after 10 epochs

# Create the learning rate scheduler callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

# Define the model checkpoint callback to save the best model weights
checkpoint_filepath = 'model_checkpoint.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Call trans.fit() with the learning rate scheduler and model checkpoint callbacks
trans.fit([x_train_ravel, pos_feed_train], y_train,
          epochs=100, batch_size=256, validation_split=0.20,
          callbacks=[lr_callback, model_checkpoint_callback])

Epoch 1/100
188/188 [==============================] - 69s 125ms/step - loss: 0.8511 - accuracy: 0.7139 - val_loss: 0.2404 - val_accuracy: 0.9254 - lr: 0.0010
Epoch 2/100
188/188 [==============================] - 21s 112ms/step - loss: 0.2161 - accuracy: 0.9328 - val_loss: 0.1842 - val_accuracy: 0.9457 - lr: 0.0010
Epoch 3/100
188/188 [==============================] - 22s 115ms/step - loss: 0.1362 - accuracy: 0.9582 - val_loss: 0.1025 - val_accuracy: 0.9682 - lr: 0.0010
Epoch 4/100
188/188 [==============================] - 20s 105ms/step - loss: 0.1046 - accuracy: 0.9672 - val_loss: 0.0960 - val_accuracy: 0.9701 - lr: 0.0010
Epoch 5/100
188/188 [==============================] - 20s 105ms/step - loss: 0.0833 - accuracy: 0.9739 - val_loss: 0.1211 - val_accuracy: 0.9636 - lr: 0.0010
Epoch 6/100
188/188 [==============================] - 20s 108ms/step - loss: 0.0725 - accuracy: 0.9769 - val_loss: 0.0823 - val_accuracy: 0.9753 - lr: 0.0010
Epoch 7/100
188/188 [=========================

In [ ]:
# saving the model for reproducibility

trans.save("/content/drive/MyDrive/transformer_5.h5")

In [ ]:
out = trans.evaluate([x_test_ravel,pos_feed_test],y_test)

313/313 [==============================] - 11s 34ms/step - loss: 0.0749 - accuracy: 0.9863


In [ ]:
out[1]

0.986299991607666

# Link to Models 5, 6, 7, 8

Since the run time was taking a long time we split the models into different Colab files and ran them on different computers/ipads/etc...

https://colab.research.google.com/drive/1Uls9-3iDvFdKDrF7_N_5n-pmvA6ZPiNA?usp=sharing

# Link to Models 9, 10, 11, 12:

https://colab.research.google.com/drive/1qRwrLwL2xPWxinZGY-N06kE7VEBm8Ji4?usp=sharing

# Link to Model 13:

https://colab.research.google.com/drive/1IO3oOYNWwE73sTsgd-pcV73u3-V5sBC0?usp=sharing